In [1]:
# Feature engineering:
# - add a number of regions where the alarm is going on
# - add number of alarms for this region during the last 24 hours

import pandas as pd

SRC_ALARMS_DATA_FOLDER = "data/source"
SRC_ALARMS_DATA_FILE = "alarms.csv"

df = pd.read_csv(f"{SRC_ALARMS_DATA_FOLDER}/{SRC_ALARMS_DATA_FILE}", delimiter=';')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19933 entries, 0 to 19932
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     19933 non-null  int64  
 1   region_id              19933 non-null  int64  
 2   region_title           19933 non-null  object 
 3   region_city            19933 non-null  object 
 4   all_region             19933 non-null  int64  
 5   start                  19933 non-null  object 
 6   end                    19933 non-null  object 
 7   clean_end              19932 non-null  object 
 8   intersection_alarm_id  2098 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 1.4+ MB


In [2]:
# define a lambda function to round up time (skip minutes and seconds part)
def round_up_time(row, column):
    # extract the date and hours parts
    dt_object = pd.to_datetime(row[column])
    time_str = dt_object.strftime("%Y-%m-%d %H:00:00")

    # return the transformed values as a dictionary
    return {column + '_rounded': time_str}

df['start_rounded'] = df.apply(lambda row: pd.Series(round_up_time(row, 'start')), axis=1)
df['end_rounded'] = df.apply(lambda row: pd.Series(round_up_time(row, 'end')), axis=1)

In [3]:
# add a new column to store the number of regions where the alarm is going on
df['num_regions'] = 0

# group the data by the start and end times of the alarms
groups = df.groupby(['start_rounded', 'end_rounded'])

# iterate over each group and count the number of unique region ids
for name, group in groups:
    num_regions = len(group['region_id'].unique())
    
    # update the corresponding rows in the dataframe with the number of regions
    df.loc[(df['start_rounded'] == name[0]) & (df['end_rounded'] == name[1]), 'num_regions'] = num_regions

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19933 entries, 0 to 19932
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     19933 non-null  int64  
 1   region_id              19933 non-null  int64  
 2   region_title           19933 non-null  object 
 3   region_city            19933 non-null  object 
 4   all_region             19933 non-null  int64  
 5   start                  19933 non-null  object 
 6   end                    19933 non-null  object 
 7   clean_end              19932 non-null  object 
 8   intersection_alarm_id  2098 non-null   float64
 9   start_rounded          19933 non-null  object 
 10  end_rounded            19933 non-null  object 
 11  num_regions            19933 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 1.8+ MB


In [5]:
# select rows where num_regions is greater than 1
filtered_df = df[df['num_regions'] > 1]

num_rows = len(filtered_df)
print("Number of rows in dataframe with num_regions > 1:", num_rows)

Number of rows in dataframe with num_regions > 1: 15974


In [6]:
# remove temp columns

df = df.drop(["start_rounded", "end_rounded"], axis=1)

In [7]:
# add feature with number of alarms for this region during the last 24 hours

# convert the 'start' and 'end' columns to datetime objects
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

# group the data by region
grouped = df.groupby('region_id')

# compute a rolling count of the number of alarms in a 24-hour window
last_24_hours = grouped.apply(lambda x: x.set_index('start').rolling('24H')['region_id'].count()).reset_index(name='num_alarms_24h')

# merge the result back into the original dataframe
df = pd.merge(df, last_24_hours, on=['region_id', 'start'], how='left')

# drop duplicates and unnecessary columns
df.drop_duplicates(subset=['region_id', 'start'], keep='last', inplace=True)
df.drop(columns=['end'], inplace=True)

In [8]:
filtered_df = df[df['num_alarms_24h'] > 1]
top_3_rows = filtered_df.head(3)
print(top_3_rows)

   id  region_id region_title region_city  all_region               start  \
4   7          3   Вінниччина     Вінниця           0 2022-02-26 08:39:39   
5   9          3   Вінниччина     Вінниця           0 2022-02-26 10:58:23   
6  10         12    Львівщина       Львів           0 2022-02-26 13:44:44   

             clean_end  intersection_alarm_id  num_regions  num_alarms_24h  
4  2022-02-26 10:42:41                    NaN            1             2.0  
5  2022-02-26 11:59:40                    NaN            1             3.0  
6  2022-02-26 14:27:25                    NaN            1             2.0  


In [9]:
# save the updated dataframe to a CSV file
OUTPUT_ALARMS_DATA_FOLDER = "data/output"
OUTPUT_ALARMS_DATA_FILE = "alarms_enriched.csv"

df.to_csv(f"{OUTPUT_ALARMS_DATA_FOLDER}/{OUTPUT_ALARMS_DATA_FILE}", index=False, sep=';')